In [ ]:
import pandas as pd
df = pd.read_csv('/content/CombinedAbstracts.csv', )

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")


In [ ]:
df=df.iloc[0:2978]
df['abstract']= df['abstract'].astype(str)
abstracts = list(nlp.pipe(df.abstract))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
#!pip install pyLDAvis
#import pyLDAvis.gensim_models
#pyLDAvis.enable_notebook()# Visualise inside a notebook
from gensim.corpora.dictionary import Dictionary
#from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

In [ ]:
tokens = []
for summary in nlp.pipe(df['abstract']):
   proj_tok = [token.lemma_.lower() for token in summary if not token.is_stop and token.is_alpha]
   tokens.append(proj_tok)
df['tokens'] = tokens
dictionary = Dictionary(df['tokens'])
print(dictionary.token2id)

In [ ]:
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(doc) for doc in df['tokens']]

In [ ]:
from gensim.models import LsiModel

In [ ]:
lsa_model = LsiModel(corpus=corpus, num_topics=10, id2word=dictionary)

In [ ]:
topics = []
score = []
for i in range(1,20,1):
   lsa_model = LsiModel(corpus=corpus, num_topics=10, id2word=dictionary)
   cm = CoherenceModel(model=lsa_model, texts = df['tokens'], corpus=corpus, dictionary=dictionary, coherence='c_v')
   topics.append(i)
   score.append(cm.get_coherence())
_=plt.plot(topics, score)
_=plt.xlabel('Number of Topics')
_=plt.ylabel('Coherence Score')
plt.show()


In [ ]:
lsa_model = LsiModel(corpus=corpus, num_topics=12, id2word=dictionary)

In [ ]:
first_doc = corpus[0]
first_doc_lsi_topics = lsa_model[first_doc]

In [ ]:
first_doc_lsi_topics

In [ ]:
all_doc_lsi_topics = lsa_model[corpus]


In [ ]:
all_doc_lsi_topics[0:2]

In [ ]:
type(lsa_model[corpus][5])

In [ ]:
def Sort_Tuple(tup):
 
    # getting length of list of tuples
    lst = len(tup)
    for i in range(0, lst):
 
        for j in range(0, lst-i-1):
            if (tup[j][1] < tup[j + 1][1]):
                temp = tup[j]
                tup[j] = tup[j + 1]
                tup[j + 1] = temp
    return tup

In [ ]:
Sort_Tuple(lsa_model[corpus][5])[1]

In [ ]:
df['topic'] = [Sort_Tuple(lsa_model[corpus][text])[1][0] for text in range(0, len(df['abstract']))]

In [ ]:
df.topic.value_counts()

In [ ]:
df2=df.groupby(['year', 'topic'])['topic'].count()

In [ ]:
df2.to_csv('lsatopics_years')

In [ ]:
df2=pd.read_csv('/content/lsatopics_years')

In [ ]:
df2.head()

In [ ]:
df3=pd.DataFrame({'Year': [
1988,
1989,
1990,
1991,
1992,
1993,
1994,
1995,
1996,
1997,
1998,
1999,
2000,
2001,
2002,
2003,
2004,
2005,
2006,
2007,
2008,
2009,
2010,
2011,
2012,
2013,
2014,
2015,
2016,
2017,
2018,
2019,
2020,
2021,
2022,
2023]})

In [ ]:
lst=[
'0',
'1',
'2',
'3',
'4',
'5',
'6',
'7',
'8',
'9',
'10',
'11']
for i in lst:
    df3[i]=[0]*36

In [ ]:
df3.head()

In [ ]:
df2.shape

In [ ]:
for i in range(373):
    a=df2.iloc[i]
    for i in range(13):
        if a.topic==i:
            y=a.year
            df3.iloc[int(y-1988),i+1]=a.count

In [ ]:
df3.head()

In [ ]:
lsa_model.show_topics(num_topics=-1, num_words=10, log=False, formatted=True)